In [1]:
import numpy as np

from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Flatten, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model

In [2]:
# load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
# from sparse label to categorical
num_labels = len(np.unique(y_train))
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [4]:
# reshape and normalize input images
image_size = x_train.shape[1]
x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
x_test = np.reshape(x_test,[-1, image_size, image_size, 1])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [5]:
x_train.shape

(60000, 28, 28, 1)

In [6]:
# network parameters
input_shape = (image_size, image_size, 1)
batch_size = 32
kernel_size = 3
dropout = 0.4
n_filters = 32

In [7]:
# left branch of Y network
left_inputs = Input(shape=input_shape)
x = left_inputs
filters = n_filters

# 3 layers of Conv2D-Dropout-MaxPooling2D
# number of filters doubles after each layer (32-64-128)
for i in range(3):
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               padding='same',
               activation='relu')(x)

    x = Dropout(dropout)(x)
    x = MaxPooling2D()(x)

    filters *= 2

In [8]:
# right branch of Y network
right_inputs = Input(shape=input_shape)
y = right_inputs
filters = n_filters

# 3 layers of Conv2D-Dropout-MaxPooling2Do
# number of filters doubles after each layer (32-64-128)
for i in range(3):
    y = Conv2D(filters=filters,
               kernel_size=kernel_size,
               padding='same',
               activation='relu',
               dilation_rate=2)(y)
    
    y = Dropout(dropout)(y)
    y = MaxPooling2D()(y)

    filters *= 2

In [9]:
# merge left and right branches outputs
y = concatenate([x, y])
# feature maps to vector before connecting to Dense
y = Flatten()(y)
y = Dropout(dropout)(y)
outputs = Dense(num_labels, activation='softmax')(y)

In [10]:
# build the model in functional API
model = Model([left_inputs, right_inputs], outputs)
# verify the model using graph
plot_model(model, to_file='./data/cnn-y-network.png', show_shapes=True)
# verify the model using layer text description
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 32)   320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 28, 28, 32)   320         input_2[0][0]                    
______________________________________________________________________________________________

In [11]:
# classifier loss, Adam optimizer, classifier accuracy
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [12]:
# train the model with input images and labels
model.fit([x_train, x_train], y_train,
          validation_data=([x_test, x_test], y_test),
          epochs=20,
          batch_size=batch_size)

Epoch 1/20
1875/1875 [==============================] - 24s 10ms/step - loss: 0.1819 - accuracy: 0.9425 - val_loss: 0.1402 - val_accuracy: 0.9883
Epoch 2/20
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0662 - accuracy: 0.9793 - val_loss: 0.1237 - val_accuracy: 0.9890
Epoch 3/20
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0523 - accuracy: 0.9840 - val_loss: 0.0672 - val_accuracy: 0.9908
Epoch 4/20
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0463 - accuracy: 0.9851 - val_loss: 0.0732 - val_accuracy: 0.9878
Epoch 5/20
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0399 - accuracy: 0.9869 - val_loss: 0.0541 - val_accuracy: 0.9915
Epoch 6/20
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0389 - accuracy: 0.9872 - val_loss: 0.0479 - val_accuracy: 0.9936
Epoch 7/20
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0362 - accuracy: 0.9886 - val_loss: 0.0505 - 

In [13]:
# model accuracy on test dataset
score = model.evaluate([x_test, x_test], y_test,
                       batch_size=batch_size,
                       verbose=0)

print("\nTest accuracy: %.1f%%" % (100.0 * score[1]))


Test accuracy: 99.2%
